In [35]:

import osmnx as ox
import folium
import pandas as pd
import numpy as np
import utils
import folium_utils
import networkx as nx
import googlemaps
import os
from dotenv import load_dotenv
import key_store
load_dotenv()

# from IPython.display import display
# pd.set_option('display.max_rows', None)

# 自宅周辺の軽いデータ                                
latitude_start = 35.330878
longitude_start = 136.951774
latitude_end = 35.402261
longitude_end = 137.072889


# latitude_start = 35.330878
# longitude_start = 136.951774
# latitude_end = 34.833082
# longitude_end = 137.672158

# キャッシュを使う
ox.config(use_cache=True, log_console=True)

print('## loading openstreetmap data')
graph = ox.graph_from_bbox(north=max(latitude_start, latitude_end),
                           south=min(latitude_start, latitude_end),
                           east=max(longitude_start, longitude_end),
                           west=min(longitude_start, longitude_end),
                           network_type='drive',
                           simplify=True,
                           retain_all=True,
                           custom_filter='["highway"~"secondary|secondary_link|primary|primary_link|trunk|trunk_link"]["lanes"=2]')
graph2 = ox.graph_from_bbox(north=max(latitude_start, latitude_end),
                           south=min(latitude_start, latitude_end),
                           east=max(longitude_start, longitude_end),
                           west=min(longitude_start, longitude_end),
                           network_type='drive',
                           simplify=True,
                           retain_all=True,
                           custom_filter='["highway"="tertiary"]')
print('## complete load openstreetmap data')
graph = nx.compose(graph, graph2)
# graph = ox.graph_from_bbox(north=max(latitude_start, latitude_end),
#                            south=min(latitude_start, latitude_end),
#                            east=max(longitude_start, longitude_end),
#                            west=min(longitude_start, longitude_end),
#                            network_type='drive',
#                            simplify=True,
#                            retain_all=True,
#                            custom_filter='["highway"~"secondary|secondary_link|primary|primary_link|trunk|trunk_link|tertiary"]')

print('## changing graph')
# グラフデータをGeoDataFrameに変換  
gdf_nodes = ox.graph_to_gdfs(graph, nodes=True, edges=False)
gdf_edges = ox.graph_to_gdfs(graph, nodes=False, edges=True)
print('## complete change graph')

# 3座標間の角度を求める
def calculate_angle_between_vectors(A, B, C):
    vector_AB = np.array(B) - np.array(A)
    vector_BC = np.array(C) - np.array(B)
    
    dot_product = np.dot(vector_AB, vector_BC)
    norm_AB = np.linalg.norm(vector_AB)
    norm_BC = np.linalg.norm(vector_BC)
    
    cosine_theta = dot_product / (norm_AB * norm_BC)
    angle_rad = np.arccos(cosine_theta)
    angle_deg = np.degrees(angle_rad)
    return angle_deg

print('## setting columns')
# 開始位置列を追加する
gdf_edges['start_point'] = gdf_edges['geometry'].apply(lambda x: x.coords[0])
# 終了位置列を追加する
gdf_edges['end_point'] = gdf_edges['geometry'].apply(lambda x: x.coords[-1])
print('## complate setting columns')

print('## deleteing reverse edge')
# 逆方向のベクトルを持つエッジを削除する
gdf_edges = utils.drop_duplicate_edge(gdf_edges)
print('## complate delete reverse edge')

print('## loading openstreetmap data for all nodes')
graph_all = ox.graph_from_bbox(north=max(latitude_start, latitude_end),
                           south=min(latitude_start, latitude_end),
                           east=max(longitude_start, longitude_end),
                           west=min(longitude_start, longitude_end),
                           network_type='drive',
                           simplify=True,
                           retain_all=True)
all_nodes = ox.graph_to_gdfs(graph_all, nodes=True, edges=False)
print('## complate load openstreetmap data for all nodes')

# エッジ内の分岐数を取得する
print('## seatching branch count')
for index, row in gdf_edges.iterrows():
  # ジオメトリーの座標と一致するノードを取得する
  nodes = all_nodes[all_nodes.geometry.intersects(row.geometry)]
  # 進行方向と逆方向のノードを除外して分岐数を計算する
  gdf_edges.at[index, 'all_street_cnt'] = nodes['street_count'].sum() - (len(nodes) * 2)
  gdf_edges.at[index, 'node_cnt'] = len(nodes)
print('## complete search branch count')

print('## calcing angle')
# 座標間の角度の変化の合計値を求める
gdf_edges['geometory_angle_total'] = gdf_edges['geometry'].apply(
    lambda x: sum([calculate_angle_between_vectors(x.coords[i-1], x.coords[i], x.coords[i+1]) for i in range(1, len(x.coords)-1)])
)
print('## complete calc angle')

print('## filtering edge')
# 以下の条件に該当するエッジを峠の候補にする
# エッジの距離が1000m以上
# ジオメトリーの座標感の角度変化の合計が120度以上
# 1500mで10分岐以下
lower_bound_meter = 1000
max_bound_meter = 10000
branch_meter_rate = 0.0066 # 1500mで10分岐以上ある場合は対象外にする
gdf_edges['is_target'] = np.where((gdf_edges['length'] >= lower_bound_meter) 
                                  & (gdf_edges['length'] < max_bound_meter)
                                  & (gdf_edges['geometory_angle_total'] > 120)
                                  & (gdf_edges['all_street_cnt'] / gdf_edges['length'] <= branch_meter_rate), 1, 0)
print('## complete filter edge')

print('## sorting edge')
# targe_edgesを並び替える
target_edges = gdf_edges[gdf_edges['is_target'] == 1]
target_edges = target_edges.sort_values(['geometory_angle_total'], ascending=[False])
print('complete sort edge')

print('## calcing elevation rate')
# 高度の変化を激しい道を優先する
# クライアントの作成
gmaps = googlemaps.Client(key=os.environ['GOOGLE_API_KEY'])
for index, row in target_edges.iterrows():
    locations = list(row.geometry.coords)
    # locationsの緯度と経度の要素番号を逆にする
    locations = [list(reversed(location)) for location in locations]
    elevations = []
    # 1度のリクエストで512箇所までしか指定できないので分割してリクエストを送る
    for i in range(0, len(locations), 512):
        # apiの呼び出し回数を減らすためにキャッシュさせる    
        parameter = locations[i:i+512]
        value = key_store.load(parameter)
        if value:
            elevations += value
        else:
            elevations.extend(gmaps.elevation(parameter))
            key_store.save(parameter, elevations)

    # elevationsのevelationの差の合計を求める
    elevation_diff_total = 0
    target_edges.at[index, 'height_diff_total'] = 0
    for i in range(1, len(elevations)):
        try:
            target_edges.at[index, 'height_diff_total'] += abs(elevations[i]['elevation'] - elevations[i-1]['elevation'])
        except Exception as e:
            # Code to handle the exception
            print(f"An exception occurred: {e}")
            print(elevations[i])
    ## 距離あたりの高低差の変化率を求める
    target_edges.at[index, 'height_diff_rate'] = target_edges.at[index, 'height_diff_total'] / row.length

print('## complete calc elevation rate')

#   # ジオメトリーの座標と一致するノードを取得する
#   nodes = all_nodes[all_nodes.geometry.intersects(row.geometry)]
#   # 進行方向と逆方向のノードを除外して分岐数を計算する
#   gdf_edges.at[index, 'all_street_cnt'] = nodes['street_count'].sum() - (len(nodes) * 2)
#   gdf_edges.at[index, 'node_cnt'] = len(nodes)

print('## calcing rate')
# height_diff_rateとheight_diff_totalの積を求める
target_edges['rate'] = target_edges['height_diff_rate'] * target_edges['geometory_angle_total']
target_edges = target_edges.sort_values(['rate'], ascending=[False])
print('## end calc rate')

print('## createing google_map_url')
target_edges['google_map'] = ''
for index, row in target_edges.iterrows():
    target_edges.at[index, 'google_map'] =  f"https://www.google.co.jp/maps/dir/{row.start_point[1]},{row.start_point[0]}/'{row.end_point[1]},{row.end_point[0]}'"
print('## complete create google_map_url')

# 地図を表示する
map_osm = ox.plot_graph_folium(graph, edge_width=2)

folium_utils.add_marker(map_osm, latitude_start, longitude_start, "st", "red")
folium_utils.add_marker(map_osm, latitude_end, longitude_end, "ed", "green")

# # is_targetが正のエッジにplotを立てて緯度と経度を表示する
# for index, row in gdf_edges[gdf_edges['is_target'] == 1].iterrows():
#     folium_utils.add_marker(map_osm,
#                             row['geometry'].coords[0][1],
#                             row['geometry'].coords[0][0],
#                             f"{row['geometry'].coords[0][1]}, {row['geometry'].coords[0][0]}, {row['geometory_angle_total']}, {row['length']}",
#                             "red")

# 候補を全て表示
map_osm.add_child(
    folium.features.GeoJson(
        gdf_edges[gdf_edges['is_target'] == 1].to_json(),
        # https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.explore.html

        style_function=lambda x: {
            'color': '#00ff00',
            'fill_opacity': 0,
            'weight': 2,
        }
    )
    .add_child(folium.features.GeoJsonPopup(
        fields=['start_point', 'end_point', 'geometory_angle_total', 'length', 'all_street_cnt', 'node_cnt'],
        aliases=['start_point', 'end_point', 'geometory_angle_total', 'length', 'all_street_cnt', 'node_cnt'],
        localize=True
    ))
)

# 候補の上位10件を表示
map_osm.add_child(
    folium.features.GeoJson(
        target_edges.head(10).to_json(),
        # https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.explore.html
        style_function=lambda x: {
            'color': '#ff0000',
            'fill_opacity': 0,
            'weight': 5,
        }
    )
    # 緯度と経度を表示する
    .add_child(folium.features.GeoJsonPopup(
        fields=['start_point', 'end_point', 'geometory_angle_total', 'length', 'all_street_cnt', 'node_cnt', 'height_diff_rate', 'height_diff_total', 'rate', 'google_map'],
        aliases=['start_point', 'end_point', 'geometory_angle_total', 'length', 'all_street_cnt', 'node_cnt', 'height_diff_rate' ,'height_diff_total', 'rate', 'google_map'],
        localize=True
    ))
)

# # 上位10件のジオメトリーをjsonで出力する
# target = target_edges
# # geometryとgeometory_angle_totalのみを抽出する
# target = target[['geometry', 'geometory_angle_total']]
# # LINESTRINGを緯度と経度のリストに変換する
# target['geometry'] = target['geometry'].apply(lambda x: list(x.coords))
# # jsonに変換する
# target[['geometry', 'geometory_angle_total']].to_json('./html/target.json', orient='records')

/tmp/ipykernel_4273/1872449440.py:30: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(use_cache=True, log_console=True)


## loading openstreetmap data
## complete load openstreetmap data
## changing graph
## complete change graph
## setting columns
## complate setting columns
## deleteing reverse edge
## complate delete reverse edge
## loading openstreetmap data for all nodes
## complate load openstreetmap data for all nodes
## seatching branch count


/tmp/ipykernel_4273/1872449440.py:109: PerformanceWarning: indexing past lexsort depth may impact performance.
  gdf_edges.at[index, 'all_street_cnt'] = nodes['street_count'].sum() - (len(nodes) * 2)
/tmp/ipykernel_4273/1872449440.py:110: PerformanceWarning: indexing past lexsort depth may impact performance.
  gdf_edges.at[index, 'node_cnt'] = len(nodes)


## complete search branch count
## calcing angle
## complete calc angle
## filtering edge
## complete filter edge
## sorting edge
complete sort edge
## calcing elevation rate
## complete calc elevation rate
## calcing rate
## end calc rate
## createing google_map_url
## complete create google_map_url


/tmp/ipykernel_4273/1872449440.py:162: PerformanceWarning: indexing past lexsort depth may impact performance.
  target_edges.at[index, 'height_diff_total'] = 0
/tmp/ipykernel_4273/1872449440.py:171: PerformanceWarning: indexing past lexsort depth may impact performance.
  target_edges.at[index, 'height_diff_rate'] = target_edges.at[index, 'height_diff_total'] / row.length
/tmp/ipykernel_4273/1872449440.py:194: UserWarning: The `folium` module has been deprecated and will be removed in a future release. You can generate and explore interactive web maps of graph nodes, edges, and/or routes automatically using GeoPandas.GeoDataFrame.explore instead, for example like: `ox.graph_to_gdfs(G, nodes=False).explore()`. See the OSMnx examples gallery for complete details and demonstrations.
  map_osm = ox.plot_graph_folium(graph, edge_width=2)
